<h1>Analysis of Bilbo C1 chip</h1>
Online analysis script used to pre-analyse the data collected during the cooldown of the chip.

In [1]:
sample_name = 'bilbo_c1'                    # name of the chip in the fridge
db_name = '20220923_sc-ncr_chip_characterisation.db'   # name of the database
user = 'sc-ncr'                               # who is performing measurements

<h3>Basic imports</h3>

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from resonator_tools import circuit

import qcodes as qc
from qcodes import initialise_database, load_by_run_spec
from qcodes.dataset.plotting import plot_by_id

import scipy.optimize
import tqdm.notebook as tqdm

db_path  = os.path.join(r'D:\QMET' + '\\' + sample_name + '\\', db_name)

<h3>Utils</h3>

In [3]:
def check_for_peaks(base_array, threshold):
    '''Detect the argmax of peaks whose value is above threshold'''
    
    # study =base_array-np.min(np.abs(base_array)) #if we want relative threshold
    study = base_array #absolute threshold
    peaks=[]
    idx=0
    L = len(study)
    while idx < L:
        ph = study[idx]
        if ph > threshold:
            if idx < L-1:
                i2 = idx+1
                ph2= ph+1
                while  i2 < L and ph2 >= ph:
                    ph2 = study[i2]
                    i2 +=1
                    
                peaks.append(idx + np.argmax(study[idx:i2])) 
                idx=i2
            else: 
                peaks.append(idx)
        idx+=1
    
    return peaks

def remove_overdetection_peaks(base_array, freq_array, peaks, minimum_freq_spacing):
    '''Remove the peaks indexes that are within minimum_freq_spacing of each other'''
    
    spacing = 0 
    grouped_peaks = [[peaks[0]]]
    for i in range(1, len(peaks)):
        pk = peaks[i]
        spacing = np.abs(freq_array[pk] - freq_array[peaks[i-1]])

        if spacing <= minimum_freq_spacing :
            grouped_peaks[-1].append(pk)
        else :
            grouped_peaks.append([pk])
            
    new_peaks = []
    for group in grouped_peaks:
        if len(group) > 1:
            n_peaks = group[0] + np.argmax(base_array[group[0]:group[-1]])
        else :
            n_peaks = group[0]
        new_peaks.append(n_peaks)
    
    return new_peaks

<h2>Data loading</h2>

In [4]:
RUN_ID = 1
qc.initialise_or_create_database_at(db_path)
dataset = qc.load_by_run_spec( captured_run_id=RUN_ID)
print(dataset)

#plt.plot(f, phi)

vna_1dtrce_p0dbm_cnt5ghz_spn3ghz #1@D:\QMET\bilbo_c1\20220923_sc-ncr_chip_characterisation.db
---------------------------------------------------------------------------------------------
ZNB20_S21_S21_frequency - numeric
ZNB20_S21_magnitude - numeric
ZNB20_S21_phase - numeric


In [5]:
db_name = '20221118_sc-ncr_s21_meas.db'
db_path  = os.path.join(r'D:\QMET' + '\\' + sample_name + '\\', db_name)

RUN_ID = 51
qc.initialise_or_create_database_at(db_path)
dataset = qc.load_by_run_spec( captured_run_id=RUN_ID)
dataset
#plt.plot(f, phi)

s21_vs_dcI_3.7Ghz_to_5.5GHz_0.1nA #51@D:\QMET\bilbo_c1\20221118_sc-ncr_s21_meas.db
----------------------------------------------------------------------------------
current_source_current - numeric
adwin_output_in_current - numeric
ZNB20_S21_S21_frequency - numeric
ZNB20_S21_magnitude - numeric
ZNB20_S21_phase - numeric

In [ ]:
f = dataset.get_parameter_data('ZNB20_S21_S21_frequency')['ZNB20_S21_S21_frequency']['ZNB20_S21_S21_frequency']
mag = dataset.get_parameter_data('ZNB20_S21_magnitude')['ZNB20_S21_magnitude']['ZNB20_S21_magnitude']
phi = dataset.get_parameter_data('ZNB20_S21_phase')['ZNB20_S21_phase']['ZNB20_S21_phase']
# pwr = dataset.get_parameter_data('mw_source_power')['mw_source_power']['mw_source_power']
# mw_f = dataset.get_parameter_data('mw_source_frequency')['mw_source_frequency']['mw_source_frequency']

In [ ]:
f = f[:mag.shape[0]]

In [ ]:
normalized_mag = mag/np.max(np.abs(mag))
# normalized_phase = np.empty(mag.shape)
# for i in range(mag.shape[0]):
#     # Normalize for each power value
#     x = mag[i, :]
#     y = phi
#     normalized_mag[i,  :] = x/np.max(np.abs(x))

<h2>Data analysis and plotting S21-Dispersion fit</h2>

In [ ]:
mag_peaks = []
phase_peaks= []
m_peaks = check_for_peaks(10*np.log(mag), threshold=-80)
m_peaks = remove_overdetection_peaks(10*np.log(mag), f, m_peaks, minimum_freq_spacing=0.03e9)
mag_peaks.append(m_peaks)
    
    # phi_peaks = check_for_peaks(phi[i, :, 0], threshold = 0)
    # phi_peaks = remove_overdetection_peaks(phi[i, :, 0], mw_f[i, :, 0], phi_peaks, minimum_freq_spacing=0.3e9)
    # phase_peaks.append(phi_peaks)

In [ ]:
fig = plt.figure(figsize=(15,10))

plt.plot(f,  10*np.log(mag))
plt.vlines(f[tuple(mag_peaks)], np.min(10*np.log(mag)), np.max(10*np.log(mag)),color='r', linestyle='--')

In [ ]:
plt.plot(f[tuple(mag_peaks)], '.')

In [ ]:

offset_K = 7*4-2

k_idx = np.arange(offset_K, offset_K + len(mag_peaks[0]), 1)
found_peaks = f[tuple(mag_peaks)]
k_idx

In [ ]:
plt.plot(k_idx, found_peaks, '.')

In [ ]:
def dispersion_fun(x, omega_p, cap_ratio, cste):
    ka = x*np.pi/2500 + cste # 2500 junctions
    
    val = omega_p*np.sqrt((1-np.cos(ka))/(1-np.cos(ka)+0.5*cap_ratio))
    return val

fit = scipy.optimize.curve_fit(dispersion_fun, k_idx, found_peaks/found_peaks[-1], bounds=(0, np.inf),  maxfev=10000 )
# print(fit[0])
plt.figure(figsize=(7,5))
plt.plot(k_idx, found_peaks, '.')
ks = np.linspace(0,k_idx[-1], num=250)
plt.plot(ks, [dispersion_fun(k, *fit[0])*found_peaks[-1] for k in ks] ,'--')
print("Plasma frequency: %.2e Hz, Cg/Cj: %.2e" % (fit[0][0]*found_peaks[-1], fit[0][1]))

<h2>Data analysis and plotting - S21 vs B field</h2>

In [ ]:
db_name = '20221118_sc-ncr_s21_meas.db'
db_path  = os.path.join(r'D:\QMET' + '\\' + sample_name + '\\', db_name)

qc.initialise_or_create_database_at(db_path)

RUN_ID = 56


dataset = qc.load_by_run_spec(captured_run_id=RUN_ID)
vnadc_vna_dict = dataset.to_pandas_dataframe_dict()
#plt.plot(f, phi)

In [ ]:

# VNA spectra
vnadc_vna_0 = vnadc_vna_dict['ZNB20_S21_magnitude']
vnadc_vna_1 = vnadc_vna_0.reset_index()

frq = vnadc_vna_1['ZNB20_S21_S21_frequency'].to_numpy()
mag = vnadc_vna_1['ZNB20_S21_magnitude'].to_numpy()
Ib = vnadc_vna_1['current_source_current'].to_numpy()

freq_size =  np.unique(frq).size
current_size = np.unique(Ib).size


m_size = (current_size, freq_size)
frq = np.reshape(frq, m_size)
mag = np.reshape(mag, m_size)
Ib = np.reshape(Ib, m_size)

In [ ]:
Ib.shape

In [ ]:
# normalized_phase = np.empty(mag.shape)
# for i in range(mag.shape[0]):
#     # Normalize for each power value
#     x = mag[i, :]
#     y = phi
#     normalized_mag[i,  :] = x/np.max(np.abs(x))

In [ ]:


mag_peaks = []
phase_peaks= []
m_peaks = check_for_peaks(10*np.log(mag[0,:]), threshold=-70)
m_peaks = remove_overdetection_peaks(10*np.log(mag[0,:]), frq[0,:], m_peaks, minimum_freq_spacing=0.04e9)
mag_peaks.append(m_peaks)
    
    # phi_peaks = check_for_peaks(phi[i, :, 0], threshold = 0)
    # phi_peaks = remove_overdetection_peaks(phi[i, :, 0], mw_f[i, :, 0], phi_peaks, minimum_freq_spacing=0.3e9)
    # phase_peaks.append(phi_peaks)

In [ ]:
fig = plt.figure(figsize=(15,10))

plt.plot(frq[0,:],  10*np.log(mag[0,:]))
plt.vlines(frq[0,tuple(mag_peaks)], np.min(10*np.log(mag[0,:])), np.max(10*np.log(mag[0,:])),color='r', linestyle='--')

In [ ]:
mag_peaks = []
phase_peaks= []
for i in range(mag.shape[0]):
    m_peaks = check_for_peaks(10*np.log(mag[i,:]), threshold=-72)
    m_peaks = remove_overdetection_peaks(10*np.log(mag[i,:]), frq[i, :], m_peaks, minimum_freq_spacing=0.03e9)
    mag_peaks.append(m_peaks)
    
    # phi_peaks = check_for_peaks(phi[i, :, 0], threshold = 2.7)
    # phi_peaks = remove_overdetection_peaks(phi[i, :], frq[i, :], phi_peaks, minimum_freq_spacing=0.5e9)
    # phase_peaks.append(phi_peaks)

In [ ]:
fig = plt.figure(figsize=(12,10))
ax1 = fig.add_subplot(111)
# ax2 = fig.add_subplot(122)


for i in range(mag.shape[0]):
    peaks =  mag_peaks[i]
    ax1.scatter(frq[i, peaks],[Ib[i, 0]]*len(peaks), color='r', s=0.5)
    
    # peaks =  phase_peaks[i]
    # ax2.scatter(frq[i, peaks, 0],[Ib[i, 0]]*len(peaks), color='r', s=0.5)

In [ ]:
fig = plt.figure(figsize=(12,10))
ax1 = fig.add_subplot(111)
# ax2 = fig.add_subplot(122)

cp = ax1.contourf(frq[:,:], Ib[:,:], 10*np.log(mag[:,:]), vmin = -110)
# cp2 = ax2.contourf(frq[:,:], Ib[:,:], phi[:,:])

for i in range(mag.shape[0]):
    peaks =  mag_peaks[i]
    ax1.scatter(frq[i, peaks],[Ib[i, 0]]*len(peaks), color='r', s=1.5)
    
    # peaks =  phase_peaks[i]
    # ax2.scatter(frq[i, peaks],[Ib[i, 0]]*len(peaks), color='r', s=1.2)

plt.colorbar(cp, ax=ax1)
# plt.colorbar(cp2, ax=ax2)

In [ ]:
## Detect the shift of a given peak

# Parameters to fiddle with
max_distance = 2e7
max_idx = 400
#######
starting_f = 4.674e9


#######
arr = np.abs( frq[0,mag_peaks[0]]- starting_f)
idx = arr.argmin()

fprevious = frq[0,mag_peaks[0]][idx]

foundX = [fprevious]
foundY = [Ib[0,0]]

for i in range(max_idx):
    peaks =  mag_peaks[i]
    fs = frq[i, peaks]
    
    arr = np.abs(fs-fprevious)
    distance = np.min(arr)
    
    if distance<max_distance:
        idx = arr.argmin()
        foundX.append(fs[idx])
        foundY.append(Ib[i,0])
        fprevious = fs[idx]

foundX = np.array(foundX)
foundY = np.array(foundY)

In [ ]:
pfit = np.polynomial.Polynomial.fit(foundY, foundX, deg=6,  domain=[Ib[0,0], Ib[-1,0]]) 
## Interpolation to measure in moving two tone

In [ ]:
fig = plt.figure(figsize=(12,10))
ax1 = fig.add_subplot(111)
# ax2 = fig.add_subplot(122)

cp = ax1.contourf(frq[:,:], Ib[:,:], 10*np.log(mag[:,:]), vmin = -110)
# cp2 = ax2.contourf(frq[:,:], Ib[:,:], phi[:,:])

ax1.scatter(foundX, foundY, color='r', s=1.5)
ax1.plot( pfit(Ib[:,0]), Ib[:,0], '--', c='darkorange')
    # peaks =  phase_peaks[i]
    # ax2.scatter(frq[i, peaks],[Ib[i, 0]]*len(peaks), color='r', s=1.2)

plt.colorbar(cp, ax=ax1)
# plt.colorbar(cp2, ax=ax2)

In [ ]:
desired_current = np.linspace(0,10e-3, num=51)
pfit(desired_current)

In [ ]:
pfit

<h2>Data analysis and plotting - Plasma (two tones) vs B field</h2>

In [ ]:
db_name = '20221118_sc-ncr_s21_meas.db'
db_path  = os.path.join(r'D:\QMET' + '\\' + sample_name + '\\', db_name)

qc.initialise_or_create_database_at(db_path)

RUN_ID = 62


dataset = qc.load_by_run_spec(captured_run_id=RUN_ID)
vnadc_vna_dict = dataset.to_pandas_dataframe_dict()
#plt.plot(f, phi)

In [ ]:

# VNA spectra
vnadc_vna_0 = vnadc_vna_dict['ZNB20_S21_magnitude']
vnadc_vna_1 = vnadc_vna_0.reset_index()

frq = vnadc_vna_1['mw_source_rs_frequency'].to_numpy()
mag = vnadc_vna_1['ZNB20_S21_magnitude'].to_numpy()
Ib = vnadc_vna_1['current_source_current'].to_numpy()

freq_size =  np.unique(frq).size
current_size = np.unique(Ib).size


m_size = (current_size, freq_size)
frq = np.reshape(frq, m_size)
mag = np.reshape(mag, m_size)
Ib = np.reshape(Ib, m_size)

In [ ]:

p_current = np.array([0,1.63,3.07,4.54,5.89,7.52,8.65,9.74,5.53,2.84,7.06])*1e-3
p_freq = np.array([9.82, 9.75,9.6,9.34,8.94,8.32,7.88,7.3,9.03,9.57,8.44])*1e9
pfit = np.polynomial.Polynomial.fit(p_current, p_freq, deg=4,  domain=[Ib[0,0], Ib[-1,0]]) 

In [ ]:
fig = plt.figure(figsize=(10,10))
ax1 = fig.add_subplot(111)

plt.interactive(True)


cp = ax1.contourf(frq[:,:], Ib[:,:], 10*np.log(mag[:,:]), vmin=-75, levels=110)


ax1.scatter(p_freq, p_current, color='r', s=1.5)
ax1.plot( pfit(Ib[:,0]), Ib[:,0], '--', c='darkorange')
plt.colorbar(cp)


In [ ]:

import sys

import logging
import numpy as np
import scipy as sp
from scipy import sparse
from scipy.sparse import linalg as splin

_has_matplotlib = True
try:
    import matplotlib.pyplot as plt
except ImportError:
    _has_matplotlib = False
    logging.warning("Matplotlib is not installed - plotting "
                    "functionality disabled")


def log_iteration(ii, s0, u, g):
    relative_change = np.linalg.norm(s0) / np.linalg.norm(u)
    g_norm = np.linalg.norm(g)
    logging.info('iteration {0:4d}: relative change = {1:.3e}, '
                 'gradient norm = {2:.3e}\n'.format(ii,
                                                    relative_change,
                                                    g_norm))


def TVRegDiff(data, itern, alph, u0=None, scale='small', ep=1e-6, dx=None,
              plotflag=_has_matplotlib, diagflag=True, precondflag=True,
              diffkernel='abs', cgtol=1e-4, cgmaxit=100):
    """
    Estimate derivatives from noisy data based using the Total 
    Variation Regularized Numerical Differentiation (TVDiff) 
    algorithm.

    Parameters
    ----------
    data : ndarray
        One-dimensional array containing series data to be
        differentiated.
    itern : int
        Number of iterations to run the main loop.  A stopping
        condition based on the norm of the gradient vector g
        below would be an easy modification.  No default value.
    alph : float    
        Regularization parameter.  This is the main parameter
        to fiddle with.  Start by varying by orders of
        magnitude until reasonable results are obtained.  A
        value to the nearest power of 10 is usally adequate.
        No default value.  Higher values increase
        regularization strenght and improve conditioning.
    u0 : ndarray, optional
        Initialization of the iteration.  Default value is the
        naive derivative (without scaling), of appropriate
        length (this being different for the two methods).
        Although the solution is theoretically independent of
        the initialization, a poor choice can exacerbate
        conditioning issues when the linear system is solved.
    scale : {large' or 'small' (case insensitive)}, str, optional   
        Default is 'small'.  'small' has somewhat better boundary
        behavior, but becomes unwieldly for data larger than
        1000 entries or so.  'large' has simpler numerics but
        is more efficient for large-scale problems.  'large' is
        more readily modified for higher-order derivatives,
        since the implicit differentiation matrix is square.
    ep : float, optional 
        Parameter for avoiding division by zero.  Default value
        is 1e-6.  Results should not be very sensitive to the
        value.  Larger values improve conditioning and
        therefore speed, while smaller values give more
        accurate results with sharper jumps.
    dx : float, optional    
        Grid spacing, used in the definition of the derivative
        operators.  Default is the reciprocal of the data size.
    plotflag : bool, optional
        Flag whether to display plot at each iteration.
        Default is True.  Useful, but adds significant
        running time.
    diagflag : bool, optional
        Flag whether to display diagnostics at each
        iteration.  Default is True.  Useful for diagnosing
        preconditioning problems.  When tolerance is not met,
        an early iterate being best is more worrying than a
        large relative residual.
    precondflag: bool, optional
        Flag whether to use a preconditioner for conjugate gradient solution.
        Default is True. While in principle it should speed things up, 
        sometimes the preconditioner can cause convergence problems instead,
        and should be turned off. Note that this mostly makes sense for 'small'
        scale problems; for 'large' ones, the improved preconditioner is one
        of the main features of the algorithms and turning it off defeats the
        point.
    diffkernel: str, optional
        Kernel to use in the integral to smooth the derivative. By default it's
        the absolute value, |u'| (value: "abs"). However, it can be changed to
        being the square, (u')^2 (value: "sq"). The latter produces smoother
        derivatives, whereas the absolute values tends to make them more blocky.
        Default is abs.
    cgtol: float, optional
        Tolerance to use in conjugate gradient optimisation. Default is 1e-4.
    cgmaxit: int, optional
        Maximum number of iterations to use in conjugate gradient optimisation. 
        Default is 100


    Returns
    -------
    u : ndarray
        Estimate of the regularized derivative of data.  Due to
        different grid assumptions, length(u) = length(data) + 1
        if scale = 'small', otherwise length(u) = length(data).
    """

    # Make sure we have a column vector
    data = np.array(data)
    assert len(data.shape) == 1, "data is not one-dimensional"
    # Get the data size.
    n = len(data)

    # Default checking. (u0 is done separately within each method.)
    if dx is None:
        dx = 1.0 / n

    # Different methods for small- and large-scale problems.
    if (scale.lower() == 'small'):

        # Differentiation operator
        d0 = -np.ones(n)/dx
        du = np.ones(n-1)/dx
        dl = np.zeros(n-1)
        dl[-1] = d0[-1]
        d0[-1] *= -1

        D = sparse.diags([dl, d0, du], [-1, 0, 1])
        DT = D.transpose()

        # Antidifferentiation and its adjoint
        def A(x): return (np.cumsum(x) - 0.5 * (x + x[0])) * dx

        def AT(x): return np.concatenate([[sum(x[1:])/2.0],
                                          (sum(x)-np.cumsum(x)+0.5*x)[1:]])*dx

        # Default initialization is naive derivative

        if u0 is None:
            u0 = D*data

        u = u0.copy()
        # Since Au( 0 ) = 0, we need to adjust.
        ofst = data[0]
        # Precompute.
        ATb = AT(ofst - data)        # input: size n

        # Main loop.
        for ii in range(1, itern+1):
            if diffkernel == 'abs':
                # Diagonal matrix of weights, for linearizing E-L equation.
                Q = sparse.spdiags(1. / (np.sqrt((D * u)**2 + ep)), 0, n, n)
                # Linearized diffusion matrix, also approximation of Hessian.
                L = dx * DT * Q * D
            elif diffkernel == 'sq':
                L = dx * DT * D
            else:
                raise ValueError('Invalid diffkernel value')

            # Gradient of functional.
            g = AT(A(u)) + ATb + alph * L * u

            # Prepare to solve linear equation.
            if precondflag:
                # Simple preconditioner.
                P = alph * sparse.spdiags(L.diagonal() + 1, 0, n, n)
            else:
                P = None

            def linop(v): return (alph * L * v + AT(A(v)))
            linop = splin.LinearOperator((n, n), linop)

            s, info_i = sparse.linalg.cg(
                linop, g, x0=None, tol=cgtol, maxiter=cgmaxit,
                callback=None, M=P, atol='legacy')

            if diagflag:
                log_iteration(ii, s[0], u, g)
                if (info_i > 0):
                    logging.warning(
                        "WARNING - convergence to tolerance not achieved!")
                elif (info_i < 0):
                    logging.warning("WARNING - illegal input or breakdown")

            # Update solution.
            u = u - s
            # Display plot.
            if plotflag:
                plt.plot(u)
                plt.show()

    elif (scale.lower() == 'large'):

        # Construct anti-differentiation operator and its adjoint.
        def A(v): return np.cumsum(v)

        def AT(w): return (sum(w) * np.ones(len(w)) -
                           np.transpose(np.concatenate(([0.0],
                                                        np.cumsum(w[:-1])))))
        # Construct differentiation matrix.
        c = np.ones(n)
        D = sparse.spdiags([-c, c], [0, 1], n, n) / dx
        mask = np.ones((n, n))
        mask[-1, -1] = 0.0
        D = sparse.dia_matrix(D.multiply(mask))
        DT = D.transpose()
        # Since Au( 0 ) = 0, we need to adjust.
        data = data - data[0]
        # Default initialization is naive derivative.
        if u0 is None:
            u0 = np.concatenate(([0], np.diff(data)))
        u = u0
        # Precompute.
        ATd = AT(data)

        # Main loop.
        for ii in range(1, itern + 1):

            if diffkernel == 'abs':
                # Diagonal matrix of weights, for linearizing E-L equation.
                Q = sparse.spdiags(1. / (np.sqrt((D * u)**2 + ep)), 0, n, n)
                # Linearized diffusion matrix, also approximation of Hessian.
                L = DT * Q * D
            elif diffkernel == 'sq':
                L = DT * D
            else:
                raise ValueError('Invalid diffkernel value')

            # Gradient of functional.
            g = AT(A(u)) - ATd
            g = g + alph * L * u
            # Build preconditioner.
            if precondflag:
                c = np.cumsum(range(n, 0, -1))
                B = alph * L + sparse.spdiags(c[::-1], 0, n, n)
                # droptol = 1.0e-2
                R = sparse.dia_matrix(np.linalg.cholesky(B.todense()))
                P = np.dot(R.transpose(), R)
            else:
                P = None
            # Prepare to solve linear equation.

            def linop(v): return (alph * L * v + AT(A(v)))
            linop = splin.LinearOperator((n, n), linop)

            s, info_i = sparse.linalg.cg(
                linop, -g, x0=None, tol=cgtol, maxiter=cgmaxit, callback=None,
                M=P, atol='legacy')
            if diagflag:
                log_iteration(ii, s[0], u, g)
                if (info_i > 0):
                    logging.warning(
                        "WARNING - convergence to tolerance not achieved!")
                elif (info_i < 0):
                    logging.warning("WARNING - illegal input or breakdown")

            # Update current solution
            u = u + s
            # Display plot
            if plotflag:
                plt.plot(u / dx)
                plt.show()

        u = u / dx

    return u


In [ ]:
def moving_average(a, n=3) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n


mag_peaks_in_freqs = []

for k in tqdm.tqdm(range(Ib.shape[0])):
    # data = i_up[k, :] - i_up[k, 0]
    data = mag[k, :]
    s_data = moving_average(data, n=50)
    s_frq= moving_average(frq[0,:], n=50)
    
    thrs = np.max(s_data)-2e-3
    
    m_peaks = check_for_peaks(np.log(s_data), threshold=-70)
    # m_peaks = remove_overdetection_peaks(s_data, s_frq, m_peaks, minimum_freq_spacing=0.1e9)
    
    mag_peaks_in_freqs.append(s_frq[m_peaks])


In [ ]:
fig = plt.figure(figsize=(12,10))
ax1 = fig.add_subplot(111)
# ax2 = fig.add_subplot(122)


for i in range(mag.shape[0]):
    peaks =  mag_peaks_in_freqs[i]
    ax1.scatter(peaks,[Ib[i, 0]]*len(peaks), color='r', s=0.5)
    
    # peaks =  phase_peaks[i]
    # ax2.scatter(frq[i, peaks, 0],[Ib[i, 0]]*len(peaks), color='r', s=0.5)

In [ ]:

data = mag[k, :]
s_data = moving_average(data, n=50)
s_frq= moving_average(frq[0,:], n=50)
gs_data = np.gradient(s_data)
plt.plot(frq[0,:],data)
plt.plot(s_frq,s_data)

In [ ]:
g_peaks = check_for_peaks(gs_data, threshold=2e-5)
g_peaks = remove_overdetection_peaks(gs_data, s_frq, g_peaks, minimum_freq_spacing=0.1e9)
plt.plot(gs_data)

# plt.vlines(g_peaks, np.min(gs_data), np.max(gs_data),color='r', linestyle='--')

In [ ]:
m_peaks = check_for_peaks(s_data, threshold=4e-3)
m_peaks = remove_overdetection_peaks(s_data, s_frq, m_peaks, minimum_freq_spacing=0.1e9)
plt.plot(s_data)

plt.vlines(m_peaks, np.min(s_data), np.max(s_data),color='r', linestyle='--')

In [ ]:

db_name = '20221118_sc-ncr_iv_curves.db'
db_path  = os.path.join(r'D:\QMET' + '\\' + sample_name + '\\', db_name)

qc.initialise_or_create_database_at(db_path)


# loading
RUN_ID = 3
dataset = qc.load_by_run_spec(captured_run_id=RUN_ID)
ivf_dict = dataset.to_pandas_dataframe_dict()

<h5>Data shaping</h5>

In [ ]:
# up ramp
ivf_up_0 = ivf_dict['adwin_up_ramp']
ivf_up_1 = ivf_up_0.reset_index()

Ib_up = ivf_up_1['current_source_current'].to_numpy()
v_up = ivf_up_1['adwin_input_2_voltage_up'].to_numpy()
i_up = ivf_up_1['adwin_up_ramp'].to_numpy()

# down ramp
ivf_down_0 = ivf_dict['adwin_down_ramp']
ivf_down_1 = ivf_down_0.reset_index()

Ib_down = ivf_down_1['current_source_current'].to_numpy()
v_down = ivf_down_1['adwin_input_2_voltage_down'].to_numpy()
i_down = ivf_down_1['adwin_down_ramp'].to_numpy()

# reshape to form a 2D array
# if RUN_ID == 4:
#     N = 20000
# if RUN_ID == 38:
#     N = 25234
# if RUN_ID == 45:
#     N = 11998    
N = np.unique(Ib_up).shape[0]

size = (N, -1)

Ib_up = np.reshape(Ib_up, size)
v_up = np.reshape(v_up, size)
i_up = np.reshape(i_up, size)
Ib_down = np.reshape(Ib_down, size)
v_down = np.reshape(v_down, size)
i_down = np.reshape(i_down, size)

# # divide into low-to-high and high-to-low
# M = len(i_up)
# zero_index_up = np.argmin(np.abs(i_up[51]))
# zero_index_down = np.argmin(np.abs(i_down[51]))

# v_lh = np.zeros((M, 2*N - zero_index_down - zero_index_up))
# v_hl = np.zeros((M, zero_index_down + zero_index_up))
# i_lh = np.zeros((M, 2*N - zero_index_down - zero_index_up))
# i_hl = np.zeros((M, zero_index_down + zero_index_up))

# for i in range(M):
#     v_lh[i] = np.concatenate(( np.flip(v_down[i][zero_index_down:]), v_up[i][zero_index_up:] ))
#     v_hl[i] = np.concatenate(( v_up[i][:zero_index_up], np.flip(v_down[i][:zero_index_down]) ))
#     i_lh[i] = np.concatenate(( np.flip(i_down[i][zero_index_down:]), i_up[i][zero_index_up:] ))
#     i_hl[i] = np.concatenate(( i_up[i][:zero_index_up], np.flip(i_down[i][:zero_index_down]) ))

<h5>Plotting</h5>

In [ ]:
# plotting
plt.rcParams.update({'font.size': 16})

plt.figure(figsize=(12,8))
voltage_scaling = 1e3
voltage_offset = 0.085
current_scaling = 1e9

h = 6.626e-34
e = 1.6e-19

for order in [1,3,9]:
    plt.plot( pfit(Ib_up[:,0])*h/2/e*voltage_scaling*order, Ib_up[:,0], '--', c='darkorange')
    boxIb = 0.0e-3+order*5e-4
    
    plt.text( pfit(boxIb)*h/2/e*voltage_scaling*order+0.008, boxIb, "n=%i" % order, size=12, bbox=dict(boxstyle="round",
                   ec=(1., 0.5, 0.5),
                   fc=(1., 0.8, 0.8),
                   alpha=0.75))

plt.imshow(current_scaling*i_up, aspect='auto', 
           extent=[ voltage_scaling*(np.min(v_up[0, :])) + voltage_offset, voltage_scaling*(np.max(v_up[0, :])) + voltage_offset, np.min(Ib_up[:, 0]), np.max(Ib_up[:,0]) ], 
           cmap='RdBu', origin='lower')
plt.xlabel('Voltage (mV)')
plt.ylabel('Coil current ($A$)')
plt.colorbar(label='DC current (nA)')
plt.savefig("plasma_scaling_vs_ivflux.png")
# plt.hlines(-1, -5, 5, linestyles='dashed', colors='black', alpha=0.5)
# plt.hlines(0, -5, 5, linestyles='dashed', colors='black', alpha=0.5)